In [3]:
# Step 0 - imports
import numpy as np
from numpy import array, inf
from numpy.random import randint, randn, seed
from math import pi
from qiskit import(QuantumCircuit, execute, Aer, IBMQ, assemble, transpile)
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.providers.ibmq.job import job_monitor
from commpy.channelcoding.convcode import Trellis, conv_encode, viterbi_decode
import hashlib, hmac
import secrets
from Crypto.Cipher import Salsa20

print("Imports Successful")

Imports Successful


In [16]:
# Step 1 - Stub definition
def kak_3_stage(message):
    prepared = alice_prepare_message(message)
    transmission_1 = alice_apply_rotation(prepared)
    transmission_2 = bob_apply_rotation(transmission_1)
    transmission_3 = alice_remove_rotation(transmission_2)
    decrypted_message = bob_remove_rotation(transmission_3)
    return decrypted_message

In [31]:
# Step 2 - External value initialization
debug = True
backend = 'qasm_simulator'
alice_key = np.random.uniform(0, 2*pi)
bob_key = np.random.uniform(0, 2*pi)
if debug:
    print("Alice's key: %s" % alice_key)
    print("Bob's key: %s" % bob_key)

Alice's key: 0.6729606938356113
Bob's key: 4.163506082841613


In [29]:
# Step 3 - Simplest possible implementations
def alice_prepare_message(message):
    if debug: print('Preparing message: %s' % message)
    qc = QuantumCircuit(1,1)
    if debug: 
        print('Alice prepared circuit')
        print(qc)
    return qc

def alice_apply_rotation(qc):
    qc.i(0)
    if debug: 
        print('After Alice applies rotation')
        print(qc)
    return qc
    
def bob_apply_rotation(qc):
    qc.i(0)
    if debug: 
        print('After Bob applies rotation')
        print(qc)
    return qc
    
def alice_remove_rotation(qc):
    qc.i(0)
    if debug: 
        print('After Alice removes rotation')
        print(qc)
    return qc

def bob_remove_rotation(qc):
    qc.i(0)
    qc.measure(0,0)
    if debug: 
        print('After Bob removes rotation and measures')
        print(qc)
    # Execute in simulator
    qasm_sim = Aer.get_backend(backend)
    qobj = assemble(qc, shots=1, memory=True)
    result = qasm_sim.run(qobj).result()
    measured_bit = int(result.get_memory()[0])
    
    return measured_bit

print(kak_3_stage(0))
print(kak_3_stage(1))

Preparing message: 0
Alice prepared circuit
     
q_0: 
     
c: 1/
     
After Alice applies rotation
     ┌───┐
q_0: ┤ I ├
     └───┘
c: 1/═════
          
After Bob applies rotation
     ┌───┐┌───┐
q_0: ┤ I ├┤ I ├
     └───┘└───┘
c: 1/══════════
               
After Alice removes rotation
     ┌───┐┌───┐┌───┐
q_0: ┤ I ├┤ I ├┤ I ├
     └───┘└───┘└───┘
c: 1/═══════════════
                    
After Bob removes rotation and measures
     ┌───┐┌───┐┌───┐┌───┐┌─┐
q_0: ┤ I ├┤ I ├┤ I ├┤ I ├┤M├
     └───┘└───┘└───┘└───┘└╥┘
c: 1/═════════════════════╩═
                          0 
0
Preparing message: 1
Alice prepared circuit
     ┌───┐
q_0: ┤ X ├
     └───┘
c: 1/═════
          
After Alice applies rotation
     ┌───┐┌───┐
q_0: ┤ X ├┤ I ├
     └───┘└───┘
c: 1/══════════
               
After Bob applies rotation
     ┌───┐┌───┐┌───┐
q_0: ┤ X ├┤ I ├┤ I ├
     └───┘└───┘└───┘
c: 1/═══════════════
                    
After Alice removes rotation
     ┌───┐┌───┐┌───┐┌───┐
q_0: ┤ X ├┤ I ├┤ I 

In [ ]:
# Step 4: NOOP as no classical component is present in this circuit

In [32]:
# Step 5: Implement real quantum portion of algorithm
# Step 3 - Simplest possible implementations
def alice_prepare_message(message):
    if debug: print('Preparing message: %s' % message)
    qc = QuantumCircuit(1,1)
    if message == 1: # Added in step 4
        qc.x(0)
    if debug: 
        print('Alice prepared circuit')
        print(qc)
    return qc

def alice_apply_rotation(qc):
    qc.rz(alice_key, 0)
    if debug: 
        print('After Alice applies rotation')
        print(qc)
    return qc
    
def bob_apply_rotation(qc):
    qc.rz(bob_key, 0)
    if debug: 
        print('After Bob applies rotation')
        print(qc)
    return qc
    
def alice_remove_rotation(qc):
    qc.rz(-alice_key, 0)
    if debug: 
        print('After Alice removes rotation')
        print(qc)
    return qc

def bob_remove_rotation(qc):
    qc.rz(-bob_key, 0)
    qc.measure(0,0)
    if debug: 
        print('After Bob removes rotation and measures')
        print(qc)
    # Execute in simulator
    qasm_sim = Aer.get_backend(backend)
    qobj = assemble(qc, shots=1, memory=True)
    result = qasm_sim.run(qobj).result()
    measured_bit = int(result.get_memory()[0])
    
    return measured_bit

print(kak_3_stage(0))
print(kak_3_stage(1))


Preparing message: 0
Alice prepared circuit
     
q_0: 
     
c: 1/
     
After Alice applies rotation
     ┌─────────────┐
q_0: ┤ RZ(0.67296) ├
     └─────────────┘
c: 1/═══════════════
                    
After Bob applies rotation
     ┌─────────────┐┌────────────┐
q_0: ┤ RZ(0.67296) ├┤ RZ(4.1635) ├
     └─────────────┘└────────────┘
c: 1/═════════════════════════════
                                  
After Alice removes rotation
     ┌─────────────┐┌────────────┐┌──────────────┐
q_0: ┤ RZ(0.67296) ├┤ RZ(4.1635) ├┤ RZ(-0.67296) ├
     └─────────────┘└────────────┘└──────────────┘
c: 1/═════════════════════════════════════════════
                                                  
After Bob removes rotation and measures
     ┌─────────────┐┌────────────┐┌──────────────┐┌─────────────┐┌─┐
q_0: ┤ RZ(0.67296) ├┤ RZ(4.1635) ├┤ RZ(-0.67296) ├┤ RZ(-4.1635) ├┤M├
     └─────────────┘└────────────┘└──────────────┘└─────────────┘└╥┘
c: 1/══════════════════════════════════════════════════════